## QHyper usecase

### Create instance of problem

In [1]:
from QHyper.problems.knapsack import KnapsackProblem
from QHyper.problems.tsp import TSPProblem

PROBLEM_TYPE = 'knapsack' # 'tsp'


# Each problem needs different parameters, because it depends on the number
# of variables and constraints
# Number of hyper_optimizer_bounds depends on the number of constraints,
# because each constraint requires one weights, and objective function also
# requires one
 
if PROBLEM_TYPE == 'knapsack':
    # Create knapsack with 3 items: 2 with weights 1 and value 2 - (1, 2),
    # and one with weight 1 and value 1 - (1, 1)
    problem = KnapsackProblem(max_weight=2, items=[(1, 2), (1, 2),(1, 1)])
    print(problem.knapsack.items)
    
    params_cofing = {
        'angles': [[0.5]*5, [1]*5],
        'hyper_args': [1, 2.5, 2.5],
    }
    hyper_optimizer_bounds = 3*[(1, 10)]
    penalty = 2

elif PROBLEM_TYPE == 'tsp':
    # Create Traveling Salesmam Problem with 3 cities
    problem = TSPProblem(
        number_of_cities=3,
    )
    params_cofing = {
        'angles': [[0.5]*5, [1]*5],
        'hyper_args': [1, 2, 2, 2, 2],
    }
    hyper_optimizer_bounds = 5*[(1, 10)]
    penalty = -1

[Item(weight=1, value=2), Item(weight=1, value=2), Item(weight=1, value=1)]


In [2]:
print(f"Variables used to describe objective function"
      f"and constraints: {problem.variables}")
print(f"Objective function: {problem.objective_function}")
print("Constraints:")
for constraint in problem.constraints:
    print(f"    {constraint}")

Variables used to describe objective functionand constraints: (x0, x1, x2, x3, x4)
Objective function: {('x0',): -2, ('x1',): -2, ('x2',): -1}
Constraints:
    {('x3',): -1, ('x4',): -1, (): 1}
    {('x0',): -1, ('x1',): -1, ('x2',): -1, ('x3',): 1, ('x4',): 2}


### Use VQA to solve knapsack problem

In [5]:
# Create a VQA instance with HQAOA as PQC and scipy optimizer
# This can be done in two various way
# 1. Providing dict with config (usefull to save experiment confing in e.g JSON)
from QHyper.solvers import Solver

solver_config = {
    "solver": {
        "type": "vqa",
        "args": {
            "config": {
                "optimizer": {
                    "type": "scipy",
                    "maxfun": 200,
                },
                "pqc": {
                    "type": "wfqaoa",
                    "layers": 5,
                    "limit_results": 20,
                    "penalty": penalty,
                    "backend": "default.qubit",
                }
            }
        }
    }
}

vqa = Solver.from_config(problem, solver_config)

# # 2. Providing actual isntance of each class like VQA and Optimizer
# from QHyper.solvers.vqa.base import VQA
# from QHyper.solvers.vqa.pqc.h_qaoa import HQAOA
# from QHyper.optimizers import ScipyOptimizer

# vqa = VQA(problem, HQAOA(layers=5), ScipyOptimizer(maxfun=200))

In [6]:
# Run VQA with provided initial parameters
solver_results = vqa.solve(params_cofing)

print("Solver results:")
print(f"Probabilities: {solver_results.results_probabilities}")
print(f"Best params: {solver_results.params}")

Solver results:
Probabilities: {'00000': 0.0732912838324004, '00001': 0.01812365507384847, '00010': 0.05407207652748644, '00011': 0.0701062245930107, '00100': 0.017498889261866067, '00101': 0.009292651366578255, '00110': 0.050293202934572685, '00111': 0.016844666311210565, '01000': 0.046624559969783416, '01001': 0.009582944060964814, '01010': 0.04374581517898236, '01011': 0.007336159588485337, '01100': 0.010700779894629564, '01101': 0.07661867874108277, '01110': 0.007507066502992263, '01111': 0.023008509073494634, '10000': 0.04662455996978336, '10001': 0.0095829440609648, '10010': 0.04374581517898232, '10011': 0.00733615958848532, '10100': 0.010700779894629532, '10101': 0.07661867874108272, '10110': 0.00750706650299228, '10111': 0.02300850907349465, '11000': 0.02873850917070903, '11001': 0.06522050516274285, '11010': 0.01700164098761874, '11011': 0.02196157538990109, '11100': 0.035843090675343484, '11101': 0.014437271029358385, '11110': 0.002494804987010662, '11111': 0.0545309266755007

In [18]:
from QHyper.util import (
    weighted_avg_evaluation, sort_solver_results, add_evaluation_to_results)

# Evaluate results with weighted average evaluation
print("Evaluation:")
print(weighted_avg_evaluation(
    solver_results.results_probabilities, problem.get_score,
    penalty=0, limit_results=10, normalize=True
))
print("Sort results:")
sorted_results = sort_solver_results(
    solver_results.results_probabilities, limit_results=10)

# Add evaluation to results
results_with_evaluation = add_evaluation_to_results(
    sorted_results, problem.get_score, penalty=penalty)

for result, (probability, evaluation) in results_with_evaluation.items():
    print(f"Result: {result}, "
          f"Prob: {probability:.5}, "
          f"Evaluation: {evaluation}")

Evaluation:
-1.2555153432454813
Sort results:
Result: 01101, Prob: 0.076619, Evaluation: -3
Result: 10101, Prob: 0.076619, Evaluation: -3
Result: 00000, Prob: 0.073291, Evaluation: 0
Result: 00011, Prob: 0.070106, Evaluation: 1
Result: 11001, Prob: 0.065221, Evaluation: -4
Result: 11111, Prob: 0.054531, Evaluation: 1
Result: 00010, Prob: 0.054072, Evaluation: 1
Result: 00110, Prob: 0.050293, Evaluation: -1
Result: 01000, Prob: 0.046625, Evaluation: 1
Result: 10000, Prob: 0.046625, Evaluation: 1


#### Using hyper optimizers

In [19]:
# Additionally other optimizer can be used to tune some parameters, in below
# example, Random optimzier will change weights (hyper_args) and choose ones
# that gives the best results after runnign 200 iteration of scipy minimizer

solver_config = {
    "solver": {
        "type": "vqa",
        "args": {
            "config": {
                "optimizer": {
                    "type": "scipy",
                    "maxfun": 200,
                },
                "pqc": {
                    "type": "wfqaoa",
                    "layers": 5,
                    "limit_results": 20,
                    "penalty": penalty,
                }
            }
        }
    },
    "hyper_optimizer": {
        "type": "random",    
        "processes": 5,
        "number_of_samples": 100,
        "bounds": hyper_optimizer_bounds,
    }
}
vqa = Solver.from_config(problem, solver_config)

In [20]:
solver_results = vqa.solve(params_cofing)

print("Solver results:")
print(f"Probabilities: {solver_results.results_probabilities}")
print(f"Best params: {solver_results.params}")

100%|██████████| 100/100 [06:00<00:00,  3.60s/it]


Solver results:
Probabilities: {'00000': 0.007118861872541281, '00001': 0.009921948041810997, '00010': 0.0015266307913184233, '00011': 0.0016639862347727916, '00100': 0.010369559565702876, '00101': 0.01806399881141381, '00110': 0.0015042657900170312, '00111': 0.005333676399055305, '01000': 0.006836296965029216, '01001': 0.011303305749643559, '01010': 0.027840715096155332, '01011': 0.009542662627372089, '01100': 0.011750509111893272, '01101': 0.02283156287760987, '01110': 0.013205497588776809, '01111': 0.01156275096899293, '10000': 0.0068362969650291976, '10001': 0.011303305749643455, '10010': 0.02784071509615512, '10011': 0.009542662627372106, '10100': 0.011750509111893247, '10101': 0.02283156287760984, '10110': 0.013205497588776802, '10111': 0.01156275096899291, '11000': 0.017692962278118122, '11001': 0.5338020887984691, '11010': 0.08673026688362817, '11011': 0.005415330576118064, '11100': 0.011769269908157047, '11101': 0.008547319581291492, '11110': 0.02784585770449222, '11111': 0.02

In [21]:
print("Evaluation:")
print(weighted_avg_evaluation(
    solver_results.results_probabilities, problem.get_score,
    penalty=0, limit_results=10, normalize=True
))
print("Sort results:")
sorted_results = sort_solver_results(
    solver_results.results_probabilities, limit_results=10)

results_with_evaluation = add_evaluation_to_results(
    sorted_results, problem.get_score, penalty=penalty)

for result, (probability, evaluation) in results_with_evaluation.items():
    print(f"Result: {result}, "
          f"Prob: {probability:.5}, "
          f"Evaluation: {evaluation}")

Evaluation:
-2.94839272145375
Sort results:
Result: 11001, Prob: 0.5338, Evaluation: -4
Result: 11010, Prob: 0.08673, Evaluation: 1
Result: 11110, Prob: 0.027846, Evaluation: 1
Result: 01010, Prob: 0.027841, Evaluation: -2
Result: 10010, Prob: 0.027841, Evaluation: -2
Result: 11111, Prob: 0.022947, Evaluation: 1
Result: 01101, Prob: 0.022832, Evaluation: -3
Result: 10101, Prob: 0.022832, Evaluation: -3
Result: 00101, Prob: 0.018064, Evaluation: 1
Result: 11000, Prob: 0.017693, Evaluation: 1
